In [25]:
import numpy as np
import scipy as sp

# Question 13

In [15]:
H = np.array([-0.8,0.2]) # high loss prob
I = np.array([0.15,0.85])
las = np.array([[1/55, 1/125],[1/80, 1/100]])
t = sp.symbols('t', positive=True)
cost_dist = 0
for i in range(2):
    for j in range(2):
        la = las[i][j]
        cost_dist+=H[i]*I[j]*la*sp.exp(-la*t)

print("cost_dist", cost_dist)
mean = sp.integrate(t*cost_dist, (t, 0, sp.oo))
var = sp.integrate(t**2*cost_dist, (t, 0, sp.oo)) - mean**2
print("var", var)
print("mean", mean)

cost_dist -0.00218181818181818*exp(-0.0181818181818182*t) + 0.000375*exp(-0.0125*t) + 0.0017*exp(-0.01*t) - 0.00544*exp(-0.008*t)
var -23404.8400000000
mean -72.2000000000000


In [17]:
import numpy as np

a = np.array([-0.00218181818181818, 0.000375, 0.0017, -0.00544])
lam = np.array([0.0181818181818182, 0.0125, 0.01, 0.008])

area = np.sum(a/lam)          # = 1.0 here
mean = np.sum(a/lam**2) / area
var = np.sum(a*2/lam**3) / area - mean**2
mean,var

(120.33333333333337, 15839.888888888887)

### Question 16

In [1]:
import numpy as np
from scipy.linalg import expm

def build_generator_Q(k: int = 12) -> np.ndarray:

    S = np.array([
        [-1.0,   1.0,  0.0,  0.0],   
        [ 0.0,  -1.0,  0.9,  0.1],   
        [ 0.0,   0.0, -1.0,  0.0],   
        [ 0.0,   0.0,  0.0, -1.0/11.0] 
    ])

    s = np.array([0.0, 0.0, 1.0, 1.0/11.0])  

    n_levels = k            
    dim = 4 * n_levels + 1    
    absorbing = dim - 1

    Q = np.zeros((dim, dim))

    def idx(phase: int, n: int) -> int:
        return 4 * n + phase

    for n in range(n_levels):
        Q[4*n:4*n+4, 4*n:4*n+4] = S

        if n < n_levels - 1:
            for phase in range(4):
                if s[phase] != 0.0:
                    Q[idx(phase, n), idx(0, n+1)] += s[phase]
        else:
            for phase in range(4):
                if s[phase] != 0.0:
                    Q[idx(phase, n), absorbing] += s[phase]

    return Q

def prob_N_less_than_k(t_months: float = 60.0, k: int = 12) -> float:

    Q = build_generator_Q(k=k)
    dim = Q.shape[0]
    absorbing = dim - 1

    p0 = np.zeros(dim)
    p0[0] = 1.0

    pt = p0 @ expm(Q * t_months)
    return 1.0 - pt[absorbing]

if __name__ == "__main__":
    t_months = 60.0  # 5 years
    k = 12   
    p = prob_N_less_than_k(t_months=t_months, k=k)
    print(f"P(N(5 years) < 12) = {p:.6f}")


P(N(5 years) < 12) = 0.189481


### Question 17

In [3]:
import scipy
S = np.array([
    [-1.0,  1.0,  0.0,  0.0],
    [ 0.0, -1.0,  0.9,  0.1],
    [ 0.0,  0.0, -1.0,  0.0],
    [ 0.0,  0.0,  0.0, -1.0/11.0]
], dtype=float)

alpha = np.array([[1.0, 0.0, 0.0, 0.0]])  # initial phase distribution for a *fresh* cycle
one = np.ones((4, 1))


inv_minus_S = np.linalg.inv(-S)
mu = float(alpha @ inv_minus_S @ one)

pi_unnormalized = alpha @ inv_minus_S
pi = pi_unnormalized / float(pi_unnormalized @ one)


t = 6.0  # months
surv_R = float(pi @ scipy.linalg.expm(S * t) @ one)          # P(R > t)
prob_within_t = 1.0 - surv_R                    # P(R <= t)

print("mu (months) =", mu)
print(f"P(next termination within {t} months) =", prob_within_t)


mu (months) = 4.0
P(next termination within 6.0 months) = 0.7887969844346261


/var/folders/g5/ds02r9td3kd859fn3kytf2nr0000gn/T/ipykernel_74938/4229827101.py:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  mu = float(alpha @ inv_minus_S @ one)
/var/folders/g5/ds02r9td3kd859fn3kytf2nr0000gn/T/ipykernel_74938/4229827101.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pi = pi_unnormalized / float(pi_unnormalized @ one)
/var/folders/g5/ds02r9td3kd859fn3kytf2nr0000gn/T/ipykernel_74938/4229827101.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1

### Question 18

We write up the generator matrix - no calculations

In [4]:
from sympy import *
l = latex(Matrix([[-1,1,0,0,0,0,0,0],[0,-1,0.9,0.1,0,0,0,0],[0.65,0,-1,0,0.35,0,0,0],[S(0.65)/11,0,0,-S(1)/11,S(0.35)/11,0,0,0],[0,0,0,0,-1,1,0,0],[0,0,0,0,0,-1,0.9,0.1],[1,0,0,0,0,0,-1,0],[S(1)/11,0,0,0,0,0,0,-S(1)/11]]))
print(l)

\left[\begin{matrix}-1 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\0 & -1 & 0.9 & 0.1 & 0 & 0 & 0 & 0\\0.65 & 0 & -1 & 0 & 0.35 & 0 & 0 & 0\\0.0590909090909091 & 0 & 0 & - \frac{1}{11} & 0.0318181818181818 & 0 & 0 & 0\\0 & 0 & 0 & 0 & -1 & 1 & 0 & 0\\0 & 0 & 0 & 0 & 0 & -1 & 0.9 & 0.1\\1 & 0 & 0 & 0 & 0 & 0 & -1 & 0\\\frac{1}{11} & 0 & 0 & 0 & 0 & 0 & 0 & - \frac{1}{11}\end{matrix}\right]


eq 1.31

In [21]:
Q = np.array([[-1,1,0,0,0],[0,-1,0.9,0.1,0],[0,0,-1,0,1],[0,0,0,-1/11,1/11],[0,0,0,0,0]])
S = Q[:-1,:-1]
U = np.linalg.inv(-S)
alpha = np.array([1,0,0,0])
one = np.ones((4,1))
t_mean = alpha @ U @ one
t_mean

array([4.])

In [30]:
1-alpha @ sp.linalg.expm(S*4) @ one

array([0.70271186])

In [43]:
S

array([[-1.        ,  1.        ,  0.        ,  0.        ],
       [ 0.        , -1.        ,  0.9       ,  0.1       ],
       [ 0.        ,  0.        , -1.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , -0.09090909]])

In [44]:
import numpy as np

S_big = np.kron(np.eye(12), S)

alpha_big = np.zeros((1, S_big.shape[0]))
one_big = np.ones((S_big.shape[0], 1))

1-alpha_big @ sp.linalg.expm(S_big*60) @ one_big

array([[1.]])

In [ ]:
1-alpha @ sp.linalg.expm(S*4) @ one